In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models
import copy
import uuid



# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imsize = 512 if torch.cuda.is_available() else 128
print(imsize)

# Image loader
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])
def image_loader(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

# Load images
image_directory = "images/"
style_img = image_loader(image_directory + "dancing.jpg")
content_img = image_loader(image_directory + "bhoomija.jpg")
assert style_img.size() == content_img.size(), "Style and content images must be the same size"

# Image display
unloader = transforms.ToPILImage()
plt.ion()
def imshow(tensor, title=None):
    image = tensor.cpu().clone().squeeze(0)
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

plt.figure()
imshow(style_img, title='Style Image')
plt.figure()
imshow(content_img, title='Content Image')

# Content loss
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        self.target = target.detach()
    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

# Gram matrix
def gram_matrix(input):
    a, b, c, d = input.size()
    features = input.view(a * b, c * d)
    G = torch.mm(features, features.t())
    return G.div(a * b * c * d)

# Style loss
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

# Load VGG-19
cnn = models.vgg19(pretrained=True).features.to(device).eval()
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# Normalization module
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)
    def forward(self, img):
        return (img - self.mean) / self.std

# Model and loss setup
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(normalization_mean, normalization_std).to(device)
    content_losses = []
    style_losses = []
    model = nn.Sequential(normalization)
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))
        model.add_module(name, layer)
        if name in content_layers:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:(i + 1)]
    return model, style_losses, content_losses

# Input image
input_img = content_img.clone()
plt.figure()
imshow(input_img, title='Input Image')

# Optimizer
def get_input_optimizer(input_img):
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

# Modified style transfer with loss and image tracking
def run_style_transfer(cnn, normalization_mean, normalization_std, content_img, style_img, input_img,
                      num_steps=300, style_weight=1000000, content_weight=1, loss_threshold=0.1,
                      display_interval=50):
    """Run the style transfer with intermediate displays, dynamic step adjustment, and loss tracking."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn, normalization_mean,
                                                                    normalization_std, style_img, content_img)
    optimizer = get_input_optimizer(input_img)

    # Lists to store losses and images
    style_losses_history = []
    content_losses_history = []
    total_losses_history = []
    intermediate_images = []
    epochs = []

    print('Optimizing..')
    run = [0]
    plt.ion()
    while run[0] <= num_steps:
        def closure():
            input_img.data.clamp_(0, 1)
            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0
            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss
            style_score *= style_weight
            content_score *= content_weight
            loss = style_score + content_score

            loss.backward()

            run[0] += 1
            if run[0] % display_interval == 0:
                # Store losses only at display intervals
                style_losses_history.append(style_score.item())
                content_losses_history.append(content_score.item())
                total_losses_history.append(loss.item())
                print(f"run {run[0]}:")
                print(f'Style Loss: {style_score.item():.4f} Content Loss: {content_score.item():.4f}')
                plt.figure()
                imshow(input_img, title=f'Output Image at Step {run[0]}')
                plt.pause(0.001)

                # Store intermediate image
                intermediate_images.append(input_img.cpu().clone())
                epochs.append(run[0])

                if loss.item() < loss_threshold:
                    print(f"Loss {loss.item():.4f} below threshold {loss_threshold}. Stopping early.")
                    return input_img, style_losses_history, content_losses_history, total_losses_history, intermediate_images, epochs

            return style_score + content_score

        optimizer.step(closure)

    input_img.data.clamp_(0, 1)
    plt.figure()
    imshow(input_img, title='Final Output Image')

    plt.ioff()
    plt.show()

    user_input = input("Is the output satisfactory? (y/n) If 'n', enter additional steps to run: ")
    if user_input.lower().startswith('n'):
        try:
            additional_steps = int(user_input[1:].strip())
            print(f"Running for {additional_steps} additional steps...")
            input_img, style_losses_history_ext, content_losses_history_ext, total_losses_history_ext, intermediate_images_ext, epochs_ext = run_style_transfer(
                cnn, normalization_mean, normalization_std, content_img, style_img, input_img,
                num_steps=additional_steps, style_weight=style_weight, content_weight=content_weight,
                loss_threshold=loss_threshold, display_interval=display_interval)
            # Extend histories
            style_losses_history.extend(style_losses_history_ext)
            content_losses_history.extend(content_losses_history_ext)
            total_losses_history.extend(total_losses_history_ext)
            intermediate_images.extend(intermediate_images_ext)
            epochs.extend([e + run[0] for e in epochs_ext])
        except ValueError:
            print("Invalid input. Exiting without additional steps.")

    return input_img, style_losses_history, content_losses_history, total_losses_history, intermediate_images, epochs

# Visualization function
def plot_losses_and_optimization(epochs, style_losses_history, content_losses_history, total_losses_history, loss_threshold):
    """Plot loss vs epochs and optimization progress."""
    plt.figure(figsize=(12, 5))

    # Plot 1: Loss vs Epochs
    plt.subplot(1, 2, 1)
    plt.plot(epochs, style_losses_history, label='Style Loss', color='blue')
    plt.plot(epochs, content_losses_history, label='Content Loss', color='green')
    plt.plot(epochs, total_losses_history, label='Total Loss', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss vs Epochs')
    plt.legend()
    plt.grid(True)

    # Plot 2: Optimization Plot (Total Loss with Annotations)
    plt.subplot(1, 2, 2)
    plt.plot(epochs, total_losses_history, label='Total Loss', color='red')
    plt.axhline(y=loss_threshold, color='gray', linestyle='--', label=f'Loss Threshold ({loss_threshold})')
    
    # Annotate key points (e.g., where loss crosses threshold)
    for i, loss in enumerate(total_losses_history):
        if i < len(epochs) and loss < loss_threshold:
            plt.annotate(f'Epoch {epochs[i]}\nLoss {loss:.2f}',
                         xy=(epochs[i], loss), xytext=(epochs[i], loss + loss_threshold * 0.1),
                         arrowprops=dict(facecolor='black', shrink=0.05))
            break
    
    plt.xlabel('Epochs')
    plt.ylabel('Total Loss')
    plt.title('Optimization Progress')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Run the style transfer
output, style_losses_history, content_losses_history, total_losses_history, intermediate_images, epochs = run_style_transfer(
    cnn, cnn_normalization_mean, cnn_normalization_std, content_img, style_img, input_img, num_steps=3000)

# Generate visualization
plot_losses_and_optimization(epochs, style_losses_history, content_losses_history, total_losses_history, loss_threshold=0.1)